In [1]:
import tensorflow.compat.v1 as tf
tf.reset_default_graph()
tf.disable_v2_behavior()
import numpy as np
import pandas as pd
import csv

Instructions for updating:
non-resource variables are not supported in the long term


In [2]:
def get_random_block_from_data(data, batch_size):
    start_index = np.random.randint(0, len(data) - batch_size)
    return data[start_index:(start_index + batch_size)]
class Autoencoder(object):

    def __init__(self, n_input, n_hidden1,n_hidden2,n_hidden3, transfer_function=tf.nn.softplus, optimizer = tf.compat.v1.train.AdamOptimizer()):
        self.n_input = n_input
        self.n_hidden1 = n_hidden1
        self.n_hidden2 = n_hidden2
        self.n_hidden3 = n_hidden3
        self.transfer = transfer_function

        network_weights = self._initialize_weights()
        self.weights = network_weights

        # model
        tf.compat.v1.disable_eager_execution()
        self.x = tf.placeholder(tf.float32, [None, self.n_input])
        self.hidden1 = self.transfer(tf.add(tf.matmul(self.x, self.weights['w1']), self.weights['b1']))
        self.hidden2 = self.transfer(tf.add(tf.matmul(self.hidden1, self.weights['w2']), self.weights['b2']))
        self.hidden3 = self.transfer(tf.add(tf.matmul(self.hidden2, self.weights['w3']), self.weights['b3']))
        self.reconstruction = tf.add(tf.matmul(self.hidden3, self.weights['w4']), self.weights['b4'])

        # cost
        self.cost = 0.5 * tf.reduce_sum(tf.pow(tf.subtract(self.reconstruction, self.x), 2.0))
        self.optimizer = optimizer.minimize(self.cost)

        init = tf.global_variables_initializer()
        self.sess = tf.Session()
        self.sess.run(init)


    def _initialize_weights(self):
        all_weights = dict()
        all_weights['w1'] = tf.get_variable("w1", shape=[self.n_input, self.n_hidden1],
            initializer=tf.glorot_uniform_initializer())
        all_weights['b1'] = tf.Variable(tf.zeros([self.n_hidden1], dtype=tf.float32))
        all_weights['w2'] = tf.get_variable("w2", shape=[self.n_hidden1, self.n_hidden2],
            initializer=tf.glorot_uniform_initializer())
        all_weights['b2'] = tf.Variable(tf.zeros([self.n_hidden2], dtype=tf.float32))
        all_weights['w3'] = tf.get_variable("w3", shape=[self.n_hidden2, self.n_hidden3],
            initializer=tf.glorot_uniform_initializer())
        all_weights['b3'] = tf.Variable(tf.zeros([self.n_hidden3], dtype=tf.float32))
        all_weights['w4'] = tf.Variable(tf.zeros([self.n_hidden3, self.n_input], dtype=tf.float32))
        all_weights['b4'] = tf.Variable(tf.zeros([self.n_input], dtype=tf.float32))
        return all_weights

    def partial_fit(self, X):
        cost, opt = self.sess.run((self.cost, self.optimizer), feed_dict={self.x: X})
        return cost

    def calc_total_cost(self, X):
        return self.sess.run(self.cost, feed_dict = {self.x: X})

    def transform(self, X):
        return self.sess.run(self.hidden2, feed_dict={self.x: X})

    def generate(self, hidden = None):
        if hidden is None:
            hidden = self.sess.run(tf.random_normal([1, self.n_hidden]))
        return self.sess.run(self.reconstruction, feed_dict={self.hidden: hidden})

    def reconstruct(self, X):
        return self.sess.run(self.reconstruction, feed_dict={self.x: X})

    def getWeights(self):
        return self.sess.run(self.weights['w1'],self.weights['w2'],self.weights['w3'],self.weights['w4'])

    def getBiases(self):
        return self.sess.run(self.weights['b1'],self.weights['b2'],self.weights['b3'],self.weights['b4'])

In [3]:
def autorunner(data_name, epochs, h1, h2, args_name):
    
    tf.reset_default_graph()
    input_path = data_name +".csv"
    X = pd.read_csv(input_path, header=None)
    #X = X.drop(0, axis=1)
    X = np.array(X)
    X = X.transpose()
    batch_size = X.shape[0]-1
    num = X.shape[1]
    file_path = args_name +".csv"
    
    n_samples,_ = np.shape(X)

    training_epochs = epochs
    display_step = 1

    autoencoder = Autoencoder(
        n_input = num,
        n_hidden1 = h1,
        n_hidden2 = h2,
        n_hidden3 = h1,
        transfer_function=tf.nn.softplus,
        optimizer=tf.train.AdamOptimizer(learning_rate=0.001))
    
    
    for epoch in range(training_epochs):
        avg_cost = 0
        total_batch = int(n_samples / batch_size)
        for i in range(total_batch):
            batch_xs = get_random_block_from_data(X, batch_size)
            cost = autoencoder.partial_fit(batch_xs)
            avg_cost += cost / n_samples * batch_size
        if epoch % display_step == 0:
            print("Epoch:", '%d,' % (epoch + 1),
                  "Cost:", "{:.9f}".format(avg_cost))    

    print("Total cost: " + str(autoencoder.calc_total_cost(X)))
    X_test_transform=autoencoder.transform(X)
    X_test_reconstruct=autoencoder.reconstruct(X)
    
    
    with open(file_path, 'w', newline='') as fout:
        writer = csv.writer(fout, delimiter=',')
        for i in X_test_reconstruct:
            writer.writerow(i)
            
    return X_test_reconstruct